# import

In [225]:
import time
import pandas as pd
from urllib.request import urlopen
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
import re

# 出馬表からレース単位で馬IDを抜き出す

In [226]:
def scrape_syutuba(race_id_list):
    data = pd.DataFrame()
    for race_id in tqdm(race_id_list):
        time.sleep(1)
        try:
            url = 'https://race.netkeiba.com/race/shutuba.html?race_id=' + race_id

            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")

            # horse_id
            horse_td_list = soup.find_all("td", attrs={'class': 'HorseInfo'})
            dfs = pd.DataFrame()
            for td in horse_td_list:
                horse_id = re.findall(r'\d+', td.find('a')['href'])[0]
                record = pd.Series([horse_id])
                dfs = dfs.append(record,ignore_index=True)

            dfs.index = [race_id] * len(dfs)
            data = data.append(dfs)
        except:
            break

    return data

# 各馬IDから近走成績を取得し、展開バイアスの公式に当てはめてカウントを行う

①近3走以内に全角4番手以内を2回以上通過した事がある馬→1でカウント
②近3走以内に全角4番手以内を1回通過した事がある馬→0.5でカウント
③近3走以内に全角4番手以内を通過した事がない馬→0でカウント（要するにカウントしない）
④逃げ馬カウント（近3走以内に2回以上逃げた馬が1頭もいない場合→－1.5カウント／近3走以内に2回以上逃げた馬が1頭だけの場合→0カウント／近3走以内に2回以上逃げた馬が2頭以上いる場合→＋1.5カウント）

In [273]:
def scrape_horse(horse_id_list):
    #逃げ馬頭数
    nigeuma = 0
    #展開バイアスカウント
    PlcOnCnt = 0.0
    #各馬情報取得
    for horse_id in tqdm(horse_id_list):
        time.sleep(1)
        try:
            url = 'https://db.netkeiba.com/horse/' + horse_id
            df = pd.read_html(url)[3]
            #受賞歴がある馬の場合4番目のデータを取得する
            if df.columns[0]=='受賞歴':
                df = pd.read_html(url)[4]
                
            required_columns = {'通過'}
            if not required_columns <= set(df.columns):
                continue
            
            #各馬近走３レース分のみ取得
            cnt = 0
            #全コーナー４番手以内回数
            Allpas4cnt = 0
            for passages in df['通過']:
                # 出走取消等は除外
                if type(passages) != str:
                    continue

                passage = passages.split('-')
                #コーナー４番手以内フラグをONにする
                Allpas4flg = True
                for pas in passage:
                    #５番手以降
                    if int(pas) > 4:
                        Allpas4flg = False
                        break
                        
                if Allpas4flg == True:
                    Allpas4cnt = Allpas4cnt + 1
                
                cnt = cnt + 1
                if cnt >= 3:
                    break
            
            #全コーナー４番手以内回数が２以上なら
            if Allpas4cnt >= 2:
                PlcOnCnt = PlcOnCnt + 1
                nigeuma = nigeuma + 1
            #全コーナー４番手以内回数が１なら
            elif Allpas4cnt == 1:
                PlcOnCnt = PlcOnCnt + 0.5
                
        except IndexError:
            continue
        except Exception as e:
            print(e)
            break
        except:
            break

    #逃げ馬の数で最終調整
    if nigeuma == 0:
        PlcOnCnt = PlcOnCnt - 1.5
    elif nigeuma >= 2:
        PlcOnCnt = PlcOnCnt + 1.5

    return PlcOnCnt

# レースIDリスト作成

In [267]:
year = 2021
course_list = [5,7,4]
kai_list = [2,3,2]
day_list = [10,6,6]
racenum = 12
race_id_list = []

for i in range(0,len(course_list),1):
    for race in range(0,racenum,1): 
        race_id = str(year).zfill(4) + str(course_list[i]).zfill(2) + str(kai_list[i]).zfill(2) + str(day_list[i]).zfill(2) +str(race+1).zfill(2)
        race_id_list.append(race_id)

# 出馬表スクレイピング

In [268]:
syutuba_list = scrape_syutuba(race_id_list)

  0%|          | 0/36 [00:00<?, ?it/s]

In [269]:
syutuba_list

,0
202105021001,2018100892
202105021001,2018101711
202105021001,2018106433
202105021001,2018105603
202105021001,2018100168
...,...
202104020612,2016101560
202104020612,2016103900
202104020612,2016101264
202104020612,2017101363


# 各馬成績スクレイピング　＆　展開バイアス算出

In [274]:
placeonbiases = []
for race_id in race_id_list:
    s_race =  syutuba_list.loc[race_id]
    placecnt = 0.0
    horse_id_list = []
    for ds,row in s_race.iterrows():
        horse_id_list.append(row[0])
    
    # 展開バイアス算出
    print(race_id + '取得中・・・')
    placecnt = scrape_horse(horse_id_list)
    print('カウント完了' + str(placecnt))
    if placecnt > 5:
        placeonbiases.append([race_id,placecnt,'×'])        
    else:
        placeonbiases.append([race_id,placecnt,'〇'])        

202105021001取得中・・・


  0%|          | 0/16 [00:00<?, ?it/s]

カウント完了5.0
202105021002取得中・・・


  0%|          | 0/16 [00:00<?, ?it/s]

カウント完了5.5
202105021003取得中・・・


...

カウント完了7.5
202104020611取得中・・・


  0%|          | 0/16 [00:00<?, ?it/s]

カウント完了11.0
202104020612取得中・・・


  0%|          | 0/15 [00:00<?, ?it/s]

カウント完了9.5


In [275]:
placeonbiases

[['202105021001', 5.0, '〇'],
 ['202105021002', 5.5, '×'],
 ['202105021003', 9.5, '×'],
 ['202105021004', 0.5, '〇'],
 ['202105021005', 9.5, '×'],
 ['202105021006', 1.5, '〇'],
 ['202105021007', 7.5, '×'],
 ['202105021008', 2.5, '〇'],
 ['202105021009', 6.5, '×'],
 ['202105021010', 9.0, '×'],
 ['202105021011', 4.5, '〇'],
 ['202105021012', 5.5, '×'],
 ['202107030601', 6.0, '×'],
 ['202107030602', 9.0, '×'],
 ['202107030603', 7.0, '×'],
 ['202107030604', 6.0, '×'],
 ['202107030605', 1.5, '〇'],
 ['202107030606', 5.5, '×'],
 ['202107030607', 5.0, '〇'],
 ['202107030608', 7.0, '×'],
 ['202107030609', 7.0, '×'],
 ['202107030610', 6.5, '×'],
 ['202107030611', 8.0, '×'],
 ['202107030612', 5.0, '〇'],
 ['202104020601', 1.0, '〇'],
 ['202104020602', 7.0, '×'],
 ['202104020603', 4.5, '〇'],
 ['202104020604', 7.5, '×'],
 ['202104020605', 8.5, '×'],
 ['202104020606', 4.5, '〇'],
 ['202104020607', 10.5, '×'],
 ['202104020608', 8.0, '×'],
 ['202104020609', 6.5, '×'],
 ['202104020610', 7.5, '×'],
 ['2021040206

# 日本語化

In [277]:
course_dict = {'01':'札幌','02':'函館','03':'福島','04':'新潟','05':'東京','06':'中山','07':'中京','08':'京都','09':'阪神','10':'小倉'}
sortlist = sorted(placeonbiases, key=lambda x: x[1])
for placeonbias in sortlist:
    rindex = placeonbias[0]
    if placeonbias[1] > 5:
        print(course_dict[rindex[4:6]] + ' ' + rindex[6:8] + '回 ' + rindex[8:10] + '日目 ' + rindex[-2:] + 'レース ' + '展開バイアスなし')
    else:
        print(course_dict[rindex[4:6]] + ' ' + rindex[6:8] + '回 ' + rindex[8:10] + '日目 ' + rindex[-2:] + 'レース ' + '展開バイアス発生：カウント' + str(placeonbias[1]))
    

東京 02回 10日目 04レース 展開バイアス発生：カウント0.5
新潟 02回 06日目 01レース 展開バイアス発生：カウント1.0
東京 02回 10日目 06レース 展開バイアス発生：カウント1.5
中京 03回 06日目 05レース 展開バイアス発生：カウント1.5
東京 02回 10日目 08レース 展開バイアス発生：カウント2.5
東京 02回 10日目 11レース 展開バイアス発生：カウント4.5
新潟 02回 06日目 03レース 展開バイアス発生：カウント4.5
新潟 02回 06日目 06レース 展開バイアス発生：カウント4.5
東京 02回 10日目 01レース 展開バイアス発生：カウント5.0
中京 03回 06日目 07レース 展開バイアス発生：カウント5.0
中京 03回 06日目 12レース 展開バイアス発生：カウント5.0
東京 02回 10日目 02レース 展開バイアスなし
東京 02回 10日目 12レース 展開バイアスなし
中京 03回 06日目 06レース 展開バイアスなし
中京 03回 06日目 01レース 展開バイアスなし
中京 03回 06日目 04レース 展開バイアスなし
東京 02回 10日目 09レース 展開バイアスなし
中京 03回 06日目 10レース 展開バイアスなし
新潟 02回 06日目 09レース 展開バイアスなし
中京 03回 06日目 03レース 展開バイアスなし
中京 03回 06日目 08レース 展開バイアスなし
中京 03回 06日目 09レース 展開バイアスなし
新潟 02回 06日目 02レース 展開バイアスなし
東京 02回 10日目 07レース 展開バイアスなし
新潟 02回 06日目 04レース 展開バイアスなし
新潟 02回 06日目 10レース 展開バイアスなし
中京 03回 06日目 11レース 展開バイアスなし
新潟 02回 06日目 08レース 展開バイアスなし
新潟 02回 06日目 05レース 展開バイアスなし
東京 02回 10日目 10レース 展開バイアスなし
中京 03回 06日目 02レース 展開バイアスなし
東京 02回 10日目 03レース 展開バイアスなし
東京 02回 10日目 05レース 展開バイアスなし
新潟 02回 06日目 12レース 展開バ